In [1]:
import keras as K
import os.path

import sys

sys.path.append("../src")
from artstat import util
import numpy as np

Using TensorFlow backend.


In [8]:
import imp
imp.reload(util)

<module 'artstat.util' from '../src/artstat/util.py'>

In [3]:
datadir = "/home/pmilovanov/hg/my/data/gallery-pr/all"

path_train = os.path.join(datadir, "train")
path_test = os.path.join(datadir, "test")

glove = "/home/pmilovanov/data/glove/glove.840B.300d.txt"

In [79]:
words, vocab = util.load_vocab("../vocab.txt", 10000)

In [10]:
emb_matrix = util.load_embeddings(vocab, 300, glove)

In [11]:
emb_matrix.shape
len(vocab)

10000

In [12]:
imp.reload(util)

seqlen = 128
stride = 64
X, Y, Xu, Yu = util.load_data_sequences(path_train, vocab, seqlen, stride)



100%|##########| 22860/22860 [00:35<00:00, 647.25it/s]


In [13]:
input_x = K.Input((seqlen,), dtype="int32", name="input_x")
input_xu = K.Input((seqlen,), dtype="float32", name="input_xu")
input_y = K.Input((seqlen,), dtype="int32", name="input_y")
input_yu = K.Input((seqlen,), dtype="float32", name="input_yu")

reshaper_u  = K.layers.Reshape((seqlen,1))
resh_xu = reshaper_u(input_xu)
resh_yu = reshaper_u(input_yu)

dim = emb_matrix.shape[1] + 1

L_emb = K.layers.Embedding(emb_matrix.shape[0], 
                           emb_matrix.shape[1], 
                           input_length=seqlen, 
                           trainable=False,
                           weights=[emb_matrix],
                           name="embedding")


In [14]:
Lemb = L_emb
emb_x = Lemb(input_x)
emb_y = Lemb(input_y)

In [15]:
Lconcat = K.layers.Concatenate()
concat_x = Lconcat([emb_x, resh_xu])
concat_y = Lconcat([emb_y, resh_yu])

In [16]:
print(concat_y.shape)
print(dense.shape)

(?, 128, 301)


NameError: name 'dense' is not defined

In [35]:
lstm = K.layers.LSTM(dim, return_sequences=True)(concat_x)
dense = K.layers.Dense(dim, activation="relu")(lstm)
dense = K.layers.Dense(dim, activation="linear")(dense)


In [36]:
reshaper = K.layers.Reshape((dim*seqlen,))
resh_x = reshaper(dense)
resh_y = reshaper(concat_y)

resh_x.shape

TensorShape([Dimension(None), Dimension(38528)])

In [37]:
proximity = K.layers.dot([resh_x, resh_y], axes=-1, normalize=True)
proximity.shape

TensorShape([Dimension(None), Dimension(1)])

In [38]:
Yfake = np.ones((Yu.shape[0], 1), dtype="float32")

In [39]:
model = K.Model(inputs=[input_x, input_xu, input_y, input_yu],
               outputs=[proximity])

In [40]:
print(Xu.shape)
print(X.shape)
print(input_x.shape)
print(input_xu.shape)

(219460, 128)
(219460, 128)
(?, 128)
(?, 128)


In [41]:
opt = K.optimizers.Adam()
model.compile(opt, loss='mean_squared_error')

In [42]:
model.fit([X, Xu, Y, Yu], [Yfake], batch_size=1024, epochs=100)


Epoch 1/100
219460/219460 [==============================] - 58s 263us/step - loss: 0.2739
Epoch 2/100
 13312/219460 [>.............................] - ETA: 55s - loss: 0.2301

KeyboardInterrupt: 

In [43]:
predictor = K.Model([input_x, input_xu], [dense])
predictor.compile(opt, loss="mean_squared_error")

In [44]:
X[3]

array([1188,   25,  285, 3817,    1,  226,   12, 2923,   11,    9,  379,
       3355,    1,  220,    3,  342,   33,  224,  446,    9, 2493,   21,
          2,  960,   24,  176,    3,    0,    3,  304,   30,  960,   14,
       5111,   21,   34, 1689,   13,  752, 4204,    1,  226,    3,    0,
         12,  941,  668,   11,   13,    7, 8687,   40,  157, 3415,   58,
          2,  695,  540,  176,    3,    0,    3,  304,    0,  512,  974,
         17,  478, 1003,    1,    0,   41,  354,    7,  270, 1981,  108,
         22,    2,    0,    4,    0,    3,  750,    2,    0,  807,  696,
       3377,    2, 2055,    4,   75,  144,  593,    3,  464,  106, 4650,
          6,    0,  776,    1,    2,  157,   10,   18,  217,  133,  858,
         16,  115, 1654,    6, 1194,    0,    3, 5988,    1,    0,  335,
          2, 3152,    4,  153, 4320, 3132,   17], dtype=int32)

In [90]:
i = 1034
tX = np.zeros((1, seqlen), dtype="int32")
tXu = np.zeros((1, seqlen), dtype="float32")
tX[0,:] = X[i]
tXu[0,:] = Xu[i]

z = predictor.predict([tX, tXu])
z

array([[[ 9.93199721e-02, -6.52906159e-03, -7.84622505e-03, ...,
         -5.51644564e-02,  1.30652949e-01,  6.00923747e-02],
        [-3.00635099e-02,  1.30821720e-01, -6.43127933e-02, ...,
         -6.18718378e-02,  8.80013257e-02,  4.89931554e-05],
        [-5.89450970e-02,  2.37443194e-01, -1.37567863e-01, ...,
         -8.95264670e-02,  1.04738146e-01, -5.66890538e-02],
        ...,
        [-8.09019804e-03,  1.23628646e-01, -9.19698104e-02, ...,
          1.86814275e-02,  6.01122305e-02,  3.45260575e-02],
        [ 2.40614116e-02,  4.66722697e-02, -1.20572783e-01, ...,
          1.05742505e-02,  7.56366923e-02,  5.17600253e-02],
        [-1.29874349e-02,  1.77095190e-01, -1.23565577e-01, ...,
         -6.06437847e-02,  6.36587366e-02, -5.04469313e-03]]],
      dtype=float32)

In [91]:
print(np.min(z), np.max(z))

-1.0330287 2.2365127


In [92]:
from sklearn import preprocessing

In [93]:
norm_emb_matrix =  preprocessing.normalize(emb_matrix, axis=1)

In [94]:
zn = preprocessing.normalize(z[0,:], axis=1)


In [95]:
we = zn[-1, :300]

In [96]:
scores = np.matmul(norm_emb_matrix, we)

In [97]:
idx = np.argmax(scores)
print(idx)
word = words[idx]

23


In [98]:
word

'at'

In [84]:
words[2]

'.'